# Install Packages

In [4]:
# ! pip -q install openai agno
# ! pip install pandas
# ! pip install python-dotenv
# ! pip install duckduckgo-search
# ! pip install -U tavily-python
# ! pip install tantivy
# ! pip install openai newspaper4k lxml_html_clean
# ! pip install pydantic-ai


  Using cached griffe-1.7.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached argcomplete-3.6.2-py3-none-any.whl.metadata (16 kB)
  Using cached mistralai-1.7.0-py3-none-any.whl.metadata (30 kB)
  Using cached openai-1.79.0-py3-none-any.whl.metadata (25 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
Using cached argcomplete-3.6.2-py3-none-any.whl (43 kB)
Using cached griffe-1.7.3-py3-none-any.whl (129 kB)
Using cached mistralai-1.7.0-py3-none-any.whl (301 kB)
Using cached openai-1.79.0-py3-none-any.whl (683 kB)
Using cached rich-14.0.0-py3-none-any.whl (243 kB)
Using cached starlette-0.46.2-py3-none-any.whl (72 kB)

   -- -------------------------------------  1/15 [griffe]
   -- -------------------------------------  1/15 [griffe]
  Attempting uninstall: starlette
   -- -------------------------------------  1/15 [griffe]
    Found existing installation: starlette 0.41.2
   -- ------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.128 requires rich<14.0.0,>=13.7.0, but you have rich 14.0.0 which is incompatible.
fastapi 0.115.9 requires starlette<0.46.0,>=0.40.0, but you have starlette 0.46.2 which is incompatible.
instructor 1.7.9 requires rich<14.0.0,>=13.7.0, but you have rich 14.0.0 which is incompatible.
litellm 1.60.2 requires httpx<0.28.0,>=0.23.0, but you have httpx 0.28.1 which is incompatible.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.1.0 which is incompatible.
streamlit 1.37.1 requires rich<14,>=10.14.0, but you have rich 14.0.0 which is incompatible.
streamlit 1.37.1 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


# Load Secrets

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path='../backend/.env')  # Adjust if not running from 'notebooks/'


# Access the variables
openai_key = os.getenv("OPENAI_API_KEY")
gmail_user = os.getenv("GMAIL_ADDRESS")
gmail_pass = os.getenv("GMAIL_APP_PASSWORD")
tavily_key = os.getenv("TAVILY_API_KEY")



print("Loaded API Key:", bool(openai_key))  # Should print True if loaded


Loaded API Key: True


# Output Schema

In [22]:
from pydantic import BaseModel
from typing import List

class AnswerSource(BaseModel):
    answer: str
    sources: List[str]

class FinalResponse(BaseModel):
    rag: AnswerSource
    web: AnswerSource
    final_summary: str


# Team Structure:

In [23]:
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel

# Initialize the model
model = OpenAIModel("gpt-4")

# RAG Agent
rag_agent = Agent(
    model=model,
    system_prompt="You are a knowledge specialist. Use the RAG system to answer queries and provide sources."
)

# Web Search Agent
web_agent = Agent(
    model=model,
    system_prompt="You are a research specialist. Conduct web searches to answer queries and provide sources."
)

# Email Agent
email_agent = Agent(
    model=model,
    system_prompt="You are an email specialist. Draft and send emails based on provided content."
)


# Team Configuration:

In [ ]:
from pydantic_ai import RunContext

# Team Lead Agent
team_lead_agent = Agent(
    model=model,
    system_prompt=(
        "You are the team lead. For any user query, delegate tasks to the RAG and Web Search agents. "
        "Combine their responses into a final summary. Then, ask the user if they would like to send this summary via email."
    ),
    output_type=FinalResponse
)


# Tools

In [ ]:
import smtplib
from email.mime.text import MIMEText
from pydantic_ai import Agent, RunContext


@team_lead_agent.tool
async def send_email(ctx: RunContext[None], content: str) -> str:
    """Send an email with the provided content using Gmail SMTP."""
    sender_email = gmail_user
    receiver_email = "nayeem60151126@gmail.com"
    password = gmail_pass  # Use an app-specific password

    message = MIMEText(content)
    message["Subject"] = "AI Summary"
    message["From"] = sender_email
    message["To"] = receiver_email

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, password)
            server.send_message(message)
        return "Email sent successfully."
    except Exception as e:
        return f"Failed to send email: {e}"


from pydantic_ai.common_tools.tavily import tavily_search_tool

# Initialize the agent with Tavily tool
team_lead_agent = Agent(
    model="openai:gpt-4",
    tools=[tavily_search_tool(api_key=tavily_key)],
    system_prompt="You are the team lead agent."
)



# Example Use

In [ ]:
import asyncio

async def main():
    user_query = "What are the latest developments in AI and LLMs?"

    # Run the Team Lead agent
    result = await team_lead_agent.run(user_query)
    output = result.output

    # Display structured outputs
    print("RAG Answer:", output.rag.answer)
    print("RAG Sources:", output.rag.sources)
    print("Web Answer:", output.web.answer)
    print("Web Sources:", output.web.sources)
    print("Final Summary:", output.final_summary)

    # Ask user for email confirmation
    confirmation = input("Would you like to send this summary via email? (yes/no): ").strip().lower()
    if confirmation == "yes":
        email_result = await send_email(None, output.final_summary)
        print(email_result)
    else:
        print("Email not sent.")

# Use this in Jupyter or notebooks
await main()


RAG Answer: AI technology is constantly evolving, and progression is being made rapidly in various fields. Here are some of the latest trends as of 2022:

1. Explainable AI: As more sectors are utilizing AI, the demand for transparency around how these systems reach their conclusions is growing. Researchers are developing "Explainable AI" that allows human users to understand and trust their decisions.

2. AI in healthcare: AI is heavily used for disease detection, drug discovery, telemedicine, patient care, etc. In the wake of the COVID-19 pandemic, AI's role in managing health data and predicting patient outcomes has seen significant progress.

3. GPT-3: OpenAI has built GPT-3, the latest version of their language prediction model, considered the most powerful language model to date.

4. Robotics Process Automation (RPA): Companies are using RPA to automate repetitive and mundane tasks, improving productivity and efficiency.

5. Quantum Machine Learning: By combining machine learning